In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup as bs

import time
from datetime import datetime

# 윈도우즈 경로
windows_path = r'C:\Users\ghdau\Desktop\real_estate_dashboard'
# 리눅스 경로로 변환
linux_path = windows_path.replace('\\', '/')

# 현재 플랫폼에 따른 파일 경로 생성
if os.name == 'nt':  # 윈도우 플랫폼
    path = linux_path
    print('Window')
else:  # 리눅스나 다른 플랫폼
    path = '/home/ubuntu/real_estate_dashboard'
    print('Linux')

# 작업 디렉토리 변경
os.chdir(path)

print(os.getcwd())

df = pd.read_csv("./0_data/legal_info_b.csv")
df

Window
C:\Users\ghdau\Desktop\real_estate_dashboard


,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


In [2]:
# 생성한 법정코드 고윳값 찾기
LAWD_CD_list = df['법정동시군구코드'].unique()
print(len(LAWD_CD_list))
LAWD_CD_list

250


array([11110, 11140, 11170, 11200, 11215, 11230, 11260, 11290, 11305,
       11320, 11350, 11380, 11410, 11440, 11470, 11500, 11530, 11545,
       11560, 11590, 11620, 11650, 11680, 11710, 11740, 26110, 26140,
       26170, 26200, 26230, 26260, 26290, 26320, 26350, 26380, 26410,
       26440, 26470, 26500, 26530, 26710, 27110, 27140, 27170, 27200,
       27230, 27260, 27290, 27710, 28110, 28140, 28177, 28185, 28200,
       28237, 28245, 28260, 28710, 28720, 29110, 29140, 29155, 29170,
       29200, 30110, 30140, 30170, 30200, 30230, 31110, 31140, 31170,
       31200, 31710, 36110, 41111, 41113, 41115, 41117, 41131, 41133,
       41135, 41150, 41171, 41173, 41190, 41210, 41220, 41250, 41271,
       41273, 41281, 41285, 41287, 41290, 41310, 41360, 41370, 41390,
       41410, 41430, 41450, 41461, 41463, 41465, 41480, 41500, 41550,
       41570, 41590, 41610, 41630, 41650, 41670, 41800, 41820, 41830,
       42110, 42130, 42150, 42170, 42190, 42210, 42230, 42720, 42730,
       42750, 42760,

### 2021년도 1월달 오피스텔 매매가 실거래가 수집을 해보겠습니다

In [3]:
# # 실행 전 시간 기록
# start_time = time.time()

# # API 키 세팅
# api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# # 컬럼
# columns = ['거래금액',
# '거래유형',
# '건축년도',
# '년',
# '단지',
# '법정동',
# '시군구',
# '월',
# '일',
# '전용면적',
# '중개사소재지',
# '지번',
# '지역코드',
# '층']

# # 서비스 url
# url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiTrade"

# # 에러 추출용
# error = []

# # 데이터 저장용
# total = pd.DataFrame()

# for i in range(len(LAWD_CD_list)):
    
#     # 데이터를 담는 리스트
#     df_total = []
    
#     # 추출하는 연월
#     year_month_key = "202101"

#     # 요청변수 파라미터 설정
#     params = "?" + \
#              "serviceKey=" + api_key + "&" + \
#              "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
#              "DEAL_YMD=" + year_month_key
    
#     res = requests.get(url + params)
#     soup = bs(res.text, 'xml')
#     items = soup.find_all('item')
    
#     if(len(items) > 0):
#         # 수집된 데이터를 데이터 프레임 형식으로 변환
#         for j in range(len(items)):
#             df_raw = []
#             for column in columns:
#                 # 컬럼 데이터가 없는 경우 처리
#                 try:
#                     df_raw.append(items[j].find(column).text)
#                 except:
#                     print(LAWD_CD_list[i], j , column) # 에러 컬럼 파악 여부
#                     df_raw.append('없음')
#             # 데이터 행 하나를 더해줍니다        
#             df_total.append(df_raw)
            
#     else:
#         # 해당 지역에 거래 데이터가 없는 경우
#         error.append(LAWD_CD_list[i])
#         continue
        
#     df = pd.DataFrame(df_total)
#     df.columns = columns
    
#     total = pd.concat([total,df])
#     total.columns = columns
    
#     print(f"{LAWD_CD_list[i]} 데이터 추출 완료")
        
        
        
        
# df = pd.read_csv("./0_data/legal_info_b.csv")      

# for i in range(len(error)):
#      print(f'{error[i]} 거래 데이터가 없습니다')
    
# # 실행 후 시간 기록
# end_time = time.time()    
    
# # 실행 시간 계산
# execution_time = end_time - start_time
# print(f"실행 시간: {execution_time} 초")   
    
# total.reset_index(drop=True,inplace=True)
# total

### 최종 파일 저장

In [4]:
# directory = '0_data/offi_trade'

# if not os.path.exists(directory):
#     os.makedirs(directory)

# file_name = f"{directory}/offi_trade_{str(year_month_key)}.csv"

# total.to_csv(file_name, index = False)

### 2021~2023 년도 모든 법정동에 대한 오피스텔 매매가 실거래가 수집

In [5]:
# API 키 세팅
api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# 컬럼
columns = ['거래금액',
'거래유형',
'건축년도',
'년',
'단지',
'법정동',
'시군구',
'월',
'일',
'전용면적',
'중개사소재지',
'지번',
'지역코드',
'층']

# 서비스 url
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiTrade"

In [6]:
years = ['2021','2022','2023']    
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
year_month = []

for year in years:
    for month in months:
        year_month.append(year+month)
print(year_month)

['202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304', '202305', '202306', '202307', '202308', '202309', '202310', '202311', '202312']


In [7]:
directory = '0_data/offi_trade'

if not os.path.exists(directory):
    os.makedirs(directory)

years = ['2021','2022','2023']    
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
year_month = []

for year in years:
    for month in months:
        year_month.append(year+month)      
    
# # 2021 연월 리스트
# year_month = ['202101','202102','202103','202104','202105','202106','202107',
#              '202108','202109','202110','202111','202112']

# 연월 반복문
for year in year_month:
    
    # 파일 저장 주소
    file_name = f"{directory}/offi_trade_{year}.csv"
    # 파일 여부 확인
    if(os.path.exists(file_name)):
       # 파일 저장 크기
        file_size = os.path.getsize(file_name)
        if(file_size > 10):
            print(f'{year} 데이터가 이미 존재합니다')
            continue
            
    print(year)
        
    # 실행 전 시간 기록
    start_time = time.time()
    
    # 연월 데이터 저장용
    total = pd.DataFrame()
    
    # 법정동 코드 반복문
    for i in range(len(LAWD_CD_list)):
    
        # 법정동 코드 별 데이터를 담는 리스트
        df_total = []

        # 요청변수 파라미터 설정
        params = "?" + \
             "serviceKey=" + api_key + "&" + \
             "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
             "DEAL_YMD=" + year

        res = requests.get(url + params)
        soup = bs(res.text, 'xml')
        items = soup.find_all('item')

        if(len(items) > 0):
            # 수집된 데이터를 데이터 프레임 형식으로 변환
            for j in range(len(items)):
                df_raw = []
                for column in columns:
                    # 컬럼 데이터가 없는 경우 처리
                    try:
                        df_raw.append(items[j].find(column).text)
                    except:
                        df_raw.append('없음')
                # 데이터 행 하나를 더해줍니다        
                df_total.append(df_raw)
        else:
            continue

        df = pd.DataFrame(df_total)
        df.columns = columns

        total = pd.concat([total,df])
        total.columns = columns
        
        print(f"{LAWD_CD_list[i]} 데이터 추출 완료")
    
    # 실행 후 시간 기록
    end_time = time.time()    
    
    # 실행 시간 계산
    execution_time = end_time - start_time
    print(f"{year} 실행 시간: {execution_time} 초")   
    
     # 파일 저장 주소
    file_name = f"{directory}/offi_trade_{str(year)}.csv"
    
    # 파일 저장
    total.to_csv(file_name, index = False)
    print(f'{file_name} 파일 저장 완료')
    print()

202101 데이터가 이미 존재합니다
202102 데이터가 이미 존재합니다
202103 데이터가 이미 존재합니다
202104 데이터가 이미 존재합니다
202105 데이터가 이미 존재합니다
202106 데이터가 이미 존재합니다
202107 데이터가 이미 존재합니다
202108 데이터가 이미 존재합니다
202109 데이터가 이미 존재합니다
202110 데이터가 이미 존재합니다
202111 데이터가 이미 존재합니다
202112 데이터가 이미 존재합니다
202201 데이터가 이미 존재합니다
202202 데이터가 이미 존재합니다
202203 데이터가 이미 존재합니다
202204 데이터가 이미 존재합니다
202205 데이터가 이미 존재합니다
202206 데이터가 이미 존재합니다
202207 데이터가 이미 존재합니다
202208 데이터가 이미 존재합니다
202209 데이터가 이미 존재합니다
202210 데이터가 이미 존재합니다
202211 데이터가 이미 존재합니다
202212 데이터가 이미 존재합니다
202301 데이터가 이미 존재합니다
202302 데이터가 이미 존재합니다
202303 데이터가 이미 존재합니다
202304 데이터가 이미 존재합니다
202305 데이터가 이미 존재합니다
202306 데이터가 이미 존재합니다
202307 데이터가 이미 존재합니다
202308 데이터가 이미 존재합니다
202309 데이터가 이미 존재합니다
202310 데이터가 이미 존재합니다
202311
11110 데이터 추출 완료
11140 데이터 추출 완료
11170 데이터 추출 완료
11200 데이터 추출 완료
11215 데이터 추출 완료
11230 데이터 추출 완료
11260 데이터 추출 완료
11290 데이터 추출 완료
11305 데이터 추출 완료
11320 데이터 추출 완료
11350 데이터 추출 완료
11380 데이터 추출 완료
11410 데이터 추출 완료
11440 데이터 추출 완료
11470 데이터 추출 완료
11500 데이터 추출 완료
11530 데이터 추출 완료
11545 데

### 최신 지도 데이터 생성

In [8]:
# API 키 세팅
api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# 컬럼
columns = ['거래금액',
'거래유형',
'건축년도',
'년',
'단지',
'법정동',
'시군구',
'월',
'일',
'전용면적',
'중개사소재지',
'지번',
'지역코드',
'층']

# 서비스 url
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiTrade"


In [9]:
# 현재 날짜 및 시간 객체 생성
now = datetime.now()

# 현재 연도와 월을 추출하여 변수에 할당
year = str(now.year)
month = str(now.month)

# 추출하는 연월
year_month_key = year + month

directory = '0_data/streamlit_data'
file_path = f"{directory}/offi_trade_{str(year_month_key)}.csv"
file_size = os.path.getsize(file_name)

# 파일이 존재하지 않으면 실행
if(os.path.exists(file_path) and file_size > 10):
    print('파일이 이미 있습니다')
    pass
else:
    # 실행 전 시간 기록
    start_time = time.time()

    # 에러 추출용
    error = []

    # 데이터 저장용
    total = pd.DataFrame()

    for i in range(len(LAWD_CD_list)):

        # 데이터를 담는 리스트
        df_total = []

        # 추출하는 연월
        year_month_key = "202311"

        # 요청변수 파라미터 설정
        params = "?" + \
                 "serviceKey=" + api_key + "&" + \
                 "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
                 "DEAL_YMD=" + year_month_key

        res = requests.get(url + params)
        soup = bs(res.text, 'xml')
        items = soup.find_all('item')

        if(len(items) > 0):
            # 수집된 데이터를 데이터 프레임 형식으로 변환
            for j in range(len(items)):
                df_raw = []
                for column in columns:
                    # 컬럼 데이터가 없는 경우 처리
                    try:
                        df_raw.append(items[j].find(column).text)
                    except:
                        print(LAWD_CD_list[i], j , column) # 에러 컬럼 파악 여부
                        df_raw.append('없음')
                # 데이터 행 하나를 더해줍니다        
                df_total.append(df_raw)

        else:
            # 해당 지역에 거래 데이터가 없는 경우
            error.append(LAWD_CD_list[i])
            continue

        df = pd.DataFrame(df_total)
        df.columns = columns

        total = pd.concat([total,df])
        total.columns = columns

        print(f"{LAWD_CD_list[i]} 데이터 추출 완료")

    df = pd.read_csv("./0_data/legal_info_b.csv")      

    for i in range(len(error)):
         print(f'{error[i]} 거래 데이터가 없습니다')

    # 실행 후 시간 기록
    end_time = time.time()    

    # 실행 시간 계산
    execution_time = end_time - start_time
    print(f"실행 시간: {execution_time} 초")   

    total.reset_index(drop=True,inplace=True)
    total

11110 데이터 추출 완료
11140 데이터 추출 완료
11170 12 건축년도
11170 데이터 추출 완료
11200 데이터 추출 완료
11215 41 건축년도
11215 데이터 추출 완료
11230 데이터 추출 완료
11260 1 건축년도
11260 데이터 추출 완료
11290 데이터 추출 완료
11305 데이터 추출 완료
11320 데이터 추출 완료
11350 데이터 추출 완료
11380 데이터 추출 완료
11410 데이터 추출 완료
11440 2 건축년도
11440 6 건축년도
11440 데이터 추출 완료
11470 데이터 추출 완료
11500 30 건축년도
11500 데이터 추출 완료
11530 데이터 추출 완료
11545 데이터 추출 완료
11560 4 건축년도
11560 35 건축년도
11560 데이터 추출 완료
11620 데이터 추출 완료
11650 35 지번
11650 36 지번
11650 데이터 추출 완료
11680 39 건축년도
11680 데이터 추출 완료
11710 3 건축년도
11710 20 건축년도
11710 23 건축년도
11710 25 건축년도
11710 26 건축년도
11710 31 건축년도
11710 33 건축년도
11710 37 건축년도
11710 42 건축년도
11710 46 건축년도
11710 데이터 추출 완료
11740 데이터 추출 완료
26110 데이터 추출 완료
26140 데이터 추출 완료
26170 데이터 추출 완료
26200 데이터 추출 완료
26230 데이터 추출 완료
26260 데이터 추출 완료
26290 데이터 추출 완료
26320 데이터 추출 완료
26350 데이터 추출 완료
26380 1 건축년도
26380 데이터 추출 완료
26410 29 건축년도
26410 데이터 추출 완료
26440 데이터 추출 완료
26470 데이터 추출 완료
26500 데이터 추출 완료
26530 데이터 추출 완료
26710 데이터 추출 완료
27110 데이터 추출 완료
27140 데이터 추출 완료
27200 데이터 추출 완료


### 결과 파일 생성

In [10]:
file_name = f"{directory}/offi_trade_{str(year_month_key)}.csv"

# 파일이 존재하지 않을 때의 조건문
if not os.path.exists(file_name):
    # 파일이 존재하지 않을 때 실행할 코드
    total.to_csv(file_name, index = False)
    print(file_name)
else:
    # 파일이 존재할 때 실행할 코드
    print("파일이 이미 존재합니다.")

파일이 이미 존재합니다.
